In [1]:
%load_ext autoreload
%aimport -tf
%aimport data
%aimport model
%autoreload 1

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import data
import model

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

In [10]:
# Hyperparameters
MAX_DOCUMENT_LENGTH = 20
VOCAB_MIN_FREQUENCY = 100
DATA_TEST_SIZE = 5000

# Model Parameters
RNN_CELL_SIZE = 128
EMBEDDING_DIM = 64

# Training Parameters
TRAIN_BATCH_SIZE = 16
TRAIN_EVAL_EVERY = 1000

In [5]:
ds = data.load_reddit_data(
    max_document_length=MAX_DOCUMENT_LENGTH,
    min_frequency=VOCAB_MIN_FREQUENCY,
    test_size=DATA_TEST_SIZE)

In [6]:
data.print_dataset_stats(ds)

Vocabulary Size: 7794
Train Data Shape: (385367, 20)
Dev Data Shape (5000, 20)


In [7]:
train_input_fn, dev_input_fn = data.create_train_dev_input_fns(ds, batch_size=TRAIN_BATCH_SIZE)

In [26]:
def simple_rnn(sequences, sequence_lengths):
    cell = tf.nn.rnn_cell.GRUCell(RNN_CELL_SIZE)
    sequence_list = tf.unpack(sequences, axis=1)
    return tf.nn.rnn(cell, sequence_list, dtype=tf.float32, sequence_length=sequence_lengths)

model_fn = model.create_lm(
    vocab_size=len(ds.vocab.vocabulary_),
    embedding_dim=EMBEDDING_DIM,
    rnn_fn=simple_rnn)

In [29]:
# Monitor for sampling sentences from the estimator
sample_mon = model.SentenceSampleMonitor(
    vocab=ds.vocab,
    every_n_steps=TRAIN_EVAL_EVERY,
    first_n_steps=-1)

# Monitor for development set loss
dev_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn=dev_input_fn,
    every_n_steps=TRAIN_EVAL_EVERY)

model_name = "maxlen_{}_rnn{}_embed_{}".format(MAX_DOCUMENT_LENGTH, RNN_CELL_SIZE, EMBEDDING_DIM)
estimator = tf.contrib.learn.Estimator(model_fn=model_fn, model_dir="./checkpoints/{}".format(model_name))

In [ ]:
estimator.fit(input_fn=train_input_fn, steps=None, monitors=[sample_mon, dev_monitor])

INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Restored model from ./checkpoints/maxlen_20_rnn128_embed_64/model.ckpt-9302-?????-of-00001


Sampling from model at ./checkpoints/maxlen_20_rnn128_embed_64/model.ckpt-9302-?????-of-00001


SETENCE_START
SETENCE_START efforts
SETENCE_START efforts Air
SETENCE_START efforts Air wings
SETENCE_START efforts Air wings nEvery
SETENCE_START efforts Air wings nEvery episodes
SETENCE_START efforts Air wings nEvery episodes Donald's
SETENCE_START efforts Air wings nEvery episodes Donald's deal
SETENCE_START efforts Air wings nEvery episodes Donald's deal sitting
SETENCE_START efforts Air wings nEvery episodes Donald's deal sitting pig
[0.00012811658, 0.00012816588, 0.00012805931, 0.0001280334, 0.00012808673, 0.00012805965, 0.00012848173, 0.00012822551, 0.00012809444, 0.00014847447]


INFO:tensorflow:Restored model from ./checkpoints/maxlen_20_rnn128_embed_64/model.ckpt-9302-?????-of-00001
INFO:tensorflow:Eval steps [0,inf) for training step 9302.
INFO:tensorflow:Results after 10 steps (0.079 sec/batch): loss = 7.16262.
INFO:tensorflow:Results after 20 steps (0.079 sec/batch): loss = 7.26977.
INFO:tensorflow:Results after 30 steps (0.077 sec/batch): loss = 7.25516.
INFO:tensorflow:Results after 40 steps (0.077 sec/batch): loss = 7.22125.
INFO:tensorflow:Results after 50 steps (0.076 sec/batch): loss = 7.37126.
INFO:tensorflow:Results after 60 steps (0.080 sec/batch): loss = 7.42404.
INFO:tensorflow:Results after 70 steps (0.078 sec/batch): loss = 7.25589.
INFO:tensorflow:Results after 80 steps (0.075 sec/batch): loss = 7.08676.
INFO:tensorflow:Results after 90 steps (0.079 sec/batch): loss = 7.23791.
INFO:tensorflow:Results after 100 steps (0.074 sec/batch): loss = 7.42615.
INFO:tensorflow:Results after 110 steps (0.080 sec/batch): loss = 7.40851.
INFO:tensorflow:Re